In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

In [2]:
tariff_list_10 = pd.read_csv("./tariff-lists/china-10-percent-030425.csv", header = None)

tariff_list_10.columns = ['line_number', 'hs-code', 'description']  

tariff_list_15 = pd.read_csv("./tariff-lists/china-15-percent-030425.csv", header = None)

tariff_list_15.columns = ['line_number', 'hs-code', 'description'] 

In [3]:
def fix_list(tariff_list):
    
    foo = tariff_list.copy(deep=True)
    
    foo["hs-code"] = foo["hs-code"].astype(str)
    
    for index, row in foo.iterrows():
        
        if len(row["hs-code"]) < 8:

            # print(row["hs-code"] )
            
            foo.loc[index, "hs-code"] = "0" + row["hs-code"]

            # print(row["hs-code"] )
            
    foo["hs6"] = foo["hs-code"].str[0:6]

    print(foo["hs6"].iloc[1:5])

    out = foo.groupby(["hs6"]).agg({"hs6":"first"})


    return out
            

In [4]:
foo_list_10 = fix_list(tariff_list_10)

foo_list_15 = fix_list(tariff_list_15)

1    100790
2    120190
3    120190
4    020311
Name: hs6, dtype: object
1    020712
2    020713
3    020713
4    020713
Name: hs6, dtype: object


In [15]:
def get_aggregate_hs6_exports(ecom):
    
    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,ALL_VAL_MO,CTY_CODE,COMM_LVL,E_COMMODITY_SDESC"
    
    surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01+to+2024-12" + "&COMM_LVL=HS6" + "&E_COMMODITY=" + ecom + "*"
    # the issue is that uniform HScodes are at the HS6 level, after that they may differ by country
    # the China ones are at HS8 and appear different, so we need to go back to the HS6 level

    url = surl + "&CTY_CODE=" + ""

    r = requests.get(url) 

    #print(r.status_code)

    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m") 

    df["exports"] = df["ALL_VAL_MO"].astype(float)

    return df[df["CTY_NAME"] == "TOTAL FOR ALL COUNTRIES"].exports.sum()

In [16]:
get_aggregate_hs6_exports("120190")

24502628814.0

In [6]:
def get_aggregate_exports():
    
    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,ALL_VAL_MO,E_COMMODITY,E_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01+to+2024-12" + "&COMM_LVL=HS2" 
    # the issue is that uniform HScodes are at the HS6 level, after that they may differ by country
    # the China ones are at HS8 and appear different, so we need to go back to the HS6 level

    url = surl + "&CTY_CODE=" + "5700"

    r = requests.get(url) 

    print(r.status_code)

    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m") 

    df["exports"] = df["ALL_VAL_MO"].astype(float)

    return df.exports.sum()

In [17]:
def get_exports_hs10(ecom, tariff):
    
    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,ALL_VAL_MO,CTY_CODE,COMM_LVL,E_COMMODITY_SDESC"
    
    surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01+to+2024-12" + "&COMM_LVL=HS6" + "&E_COMMODITY=" + ecom + "*"
    # the issue is that uniform HScodes are at the HS6 level, after that they may differ by country
    # the China ones are at HS8 and appear different, so we need to go back to the HS6 level

    url = surl + "&CTY_CODE=" + "5700"

    # China is 5700
    
    r = requests.get(url) 
    
    #print(r.status_code)
    
    if r.status_code == 200:
    
        df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe

        df.columns = r.json()[0]

        df.time = pd.to_datetime(df.time, format="%Y-%m") 

        df["exports"] = df["ALL_VAL_MO"].astype(float)
        
        df["description"] = df["E_COMMODITY_SDESC"]
            
        df["hs6"] = ecom
        # Now there whould only be one hs code per call, but I'm going use the groupby to be safe

        df.drop(["ALL_VAL_MO", "E_COMMODITY", "E_COMMODITY_SDESC", "COMM_LVL"], axis = 1, inplace = True)
        
        grp = df.groupby(["hs6"]) # group all exports over all the months (again there should only be one hs6 code)

        top_products = grp.agg({"exports":"sum", "description":"first"})
               
        top_products["hs-code"] = ecom
        
        top_products["tariff"] = tariff

        top_products["export_share"] = 100.*( top_products["exports"] / get_aggregate_hs6_exports(ecom) )

        top_products["color"] = "#EE1C25"
    
        return top_products
    
    else:
        # some of the products have no trade, this just deals with these issues. 
    
        df = pd.DataFrame(columns=['exports', 'description', 'hs-code', 'tariff', 'export_share', 'color'])
        
        df.index.name = "hs6"
        
        new_row = {'exports': 0.0, 'description': " ", 'hs-code': ecom, 'tariff': tariff, 'export_share': 0.0, 'color': "#EE1C25"}
        # so if its empty, we just create a row with zero exports and the description comes from the
        # chinese side
        
        df = pd.concat([df, pd.DataFrame([new_row], index=[ecom])])
    
    return df

In [18]:
foo = get_exports_hs10("120190", 10.0)

In [20]:
# this will then work through the tariff list and construct the data frame

trade_df_10 = pd.DataFrame(columns=['exports',"description",'hs-code'])

tariff_rate = 10.0

for index, row in foo_list_10.iterrows():
    
#     print(index)
    
#     print(row['hs-code'])
    
    foo = get_exports_hs10(row['hs6'], tariff_rate)
    
    trade_df_10 = pd.concat([trade_df_10, foo])

In [22]:
trade_df_10.sort_values(by = ["exports"], ascending = False)[0:20]

,exports,description,hs-code,tariff,export_share,color
120190,1.276137e+10,"SOYBEANS, NESOI",120190,10.0,52.081636,#EE1C25
100790,1.259120e+09,"GRAIN SORGHUM, NESOI",100790,10.0,89.081259,#EE1C25
020230,9.998870e+08,"MEAT OF BOVINE ANIMALS, BONELESS, FROZEN",020230,10.0,27.025471,#EE1C25
080251,6.277297e+08,"PISTACHIOS, IN SHELL, FRESH OR DRIED",080251,10.0,27.338522,#EE1C25
020649,5.935177e+08,"OFFAL OF SWINE EXCEPT LIVERS, EDIBLE, FROZEN",020649,10.0,72.822148,#EE1C25
020329,3.106005e+08,"MEAT OF SWINE, NESOI, FROZEN",020329,10.0,9.921790,#EE1C25
020130,3.052876e+08,"MEAT OF BOVINE ANIMALS, BONELESS, FRESH OR CHI...",020130,10.0,7.377984,#EE1C25
200819,2.100323e+08,"NUTS (EXC PEANUTS) AND SEEDS, PREPARED ETC. NESOI",200819,10.0,22.039849,#EE1C25
020220,1.880347e+08,"MEAT, BOVINE CUTS WITH BONE IN, FROZEN",020220,10.0,19.405136,#EE1C25
350220,1.568946e+08,"MILK ALBUMIN,INC CONCEN OF 2 OR MORE WHEY PROT...",350220,10.0,19.421351,#EE1C25


In [23]:
trade_df_10.exports.sum()

19045437268.0

In [24]:
trade_df_10 = trade_df_10.astype({
    'hs-code': 'string',
})

pq.write_table(pa.Table.from_pandas(trade_df_10), "./data/china-10-percent-US-exports-030425.parquet")

trade_df_10.to_csv("./data/china-10-percent-US-exports-030425.csv", index=False)

In [91]:
trade_df_10.dtypes

exports        float64
description     object
hs-code         string
tariff         float64
dtype: object

In [25]:
# this will then work through the tariff list and construct the data frame

trade_df_15 = pd.DataFrame(columns=['exports',"description",'hs-code'])

tariff_rate = 15.0

for index, row in foo_list_15.iterrows():
    
#     print(index)
    
#     print(row['hs-code'])
    
    foo = get_exports_hs10(row['hs6'], tariff_rate)
    
    trade_df_15 = pd.concat([trade_df_15, foo])

In [26]:
trade_df_15.sort_values(by = ["exports"], ascending = False)[0:30]

,exports,description,hs-code,tariff,export_share,color
520100,1.487518e+09,"COTTON, NOT CARDED OR COMBED",520100,15.0,29.703900,#EE1C25
100199,5.569497e+08,"WHEAT AND MESLIN, NESOI",100199,15.0,9.683841,#EE1C25
020714,3.399914e+08,"CHICKEN CUTS AND EDIBLE OFFAL (INC LIVERS), FR...",020714,15.0,10.407037,#EE1C25
100590,3.309318e+08,"CORN (MAIZE), OTHER THAN SEED CORN",100590,15.0,2.354371,#EE1C25
160232,1.379402e+08,"PREPARED OR PRESERVED CHICKEN MEAT OR OFFAL, N...",160232,15.0,33.528331,#EE1C25
100191,2.950891e+06,"WHEAT AND MESLIN SEED, NESOI",100191,15.0,25.906791,#EE1C25
110100,5.734950e+05,WHEAT OR MESLIN FLOUR,110100,15.0,0.379538,#EE1C25
520300,3.420600e+05,"COTTON, CARDED OR COMBED",520300,15.0,4.022376,#EE1C25
020713,9.396200e+04,CHICKEN CUTS & EDIBLE OFFAL (INCL LIVER) FRSH/...,020713,15.0,0.009257,#EE1C25
020712,7.281400e+04,"MEAT & OFFAL OF CHICKENS,NOT CUT IN PIECES,FROZEN",020712,15.0,0.420715,#EE1C25


In [28]:
trade_df_15.exports.sum()

trade_df_15 = trade_df_15.astype({
    'hs-code': 'string',
})

pq.write_table(pa.Table.from_pandas(trade_df_15), "./data/china-15-percent-US-exports-030425.parquet")

trade_df_15.to_csv("./data/china-15-percent-US-exports-030425.csv", index=False)

In [27]:
trade_df_15.exports.sum()

2857368987.0

In [29]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["exports"]/1000000000).map('{:,.1f}'.format)
    
    df["hover_label_2"] = (df["export_share"]).map('{:,.1f}'.format)
    
    df["hover_label_3"] = (df["tariff"]).map('{:,.1f}'.format)
    
    source = ColumnDataSource(df)
    
    return source

In [60]:
def make_bar_chart(df):

    width = 600
    height = 500

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "Top US Exports to China Subject to Retaliatory Tariffs",
           toolbar_location = 'below',
           tools = "reset")
        
    p.vbar(x = "position", top = "exports", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "color",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@description</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Exports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Share of Total: @hover_label_2%</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff: @hover_label_3%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '13pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

    p.yaxis.formatter = NumeralTickFormatter(format="($0.0 a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df.exports.max() + 0.10*df.exports.max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)
    return p

In [61]:
p1 = make_bar_chart(trade_df_10.sort_values(by = ["exports"], ascending = False)[0:20])

p2 = make_bar_chart(trade_df_15.sort_values(by = ["exports"], ascending = False)[0:10])

In [62]:
tab1 = Panel(child= p1, title="10% Tariff")

tab2 = Panel(child= p2, title="15% Tariff")

output_file('.\\docs\\' + "china-retaliation.html")

div0 = Div(text = """Each bar represents the total sum of U.S. exports in 2024 for a HS6 category subject to Chinese tariffs. 
Only max 20 categories are displayed. Hover your cursor over each bar to learn more.
""", max_width=600, background = background )

div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab1, tab2], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)

In [30]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"